In [29]:
import pandas as pd
import seaborn as sb
import numpy as np
from sklearn.preprocessing import LabelEncoder

import geocoder

pd.options.display.float_format = "{:.2f}".format

In [2]:
yel2019 = pd.read_csv("/Users/james.fung/Desktop/test_scripts/via/yellow_tripdata_2016-01.csv")

In [3]:
zones = pd.read_csv("/Users/james.fung/Desktop/test_scripts/via/taxi_zones.csv")

In [4]:
boros = pd.read_csv("/Users/james.fung/Downloads/nybb (1).csv")

In [8]:
yel2019.head(10)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-01-01 00:00:00,2016-01-01 00:00:00,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,7.5,0.5,0.5,0.0,0.0,0.3,8.8
1,2,2016-01-01 00:00:00,2016-01-01 00:00:00,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,18.0,0.5,0.5,0.0,0.0,0.3,19.3
2,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,33.0,0.5,0.5,0.0,0.0,0.3,34.3
3,2,2016-01-01 00:00:00,2016-01-01 00:00:00,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,16.5,0.0,0.5,0.0,0.0,0.3,17.3
4,2,2016-01-01 00:00:00,2016-01-01 00:00:00,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,8.0,0.0,0.5,0.0,0.0,0.3,8.8
5,2,2016-01-01 00:00:00,2016-01-01 00:18:30,2,5.52,-73.980118,40.743050,1,N,-73.913490,40.763142,2,19.0,0.5,0.5,0.0,0.0,0.3,20.3
6,2,2016-01-01 00:00:00,2016-01-01 00:26:45,2,7.45,-73.994057,40.719990,1,N,-73.966362,40.789871,2,26.0,0.5,0.5,0.0,0.0,0.3,27.3
7,1,2016-01-01 00:00:01,2016-01-01 00:11:55,1,1.20,-73.979424,40.744614,1,N,-73.992035,40.753944,2,9.0,0.5,0.5,0.0,0.0,0.3,10.3
8,1,2016-01-01 00:00:02,2016-01-01 00:11:14,1,6.00,-73.947151,40.791046,1,N,-73.920769,40.865578,2,18.0,0.5,0.5,0.0,0.0,0.3,19.3
9,2,2016-01-01 00:00:02,2016-01-01 00:11:08,1,3.21,-73.998344,40.723896,1,N,-73.995850,40.688400,2,11.5,0.5,0.5,0.0,0.0,0.3,12.8


In [9]:
yel2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10906858 entries, 0 to 10906857
Data columns (total 19 columns):
VendorID                 int64
tpep_pickup_datetime     object
tpep_dropoff_datetime    object
passenger_count          int64
trip_distance            float64
pickup_longitude         float64
pickup_latitude          float64
RatecodeID               int64
store_and_fwd_flag       object
dropoff_longitude        float64
dropoff_latitude         float64
payment_type             int64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
improvement_surcharge    float64
total_amount             float64
dtypes: float64(12), int64(4), object(3)
memory usage: 1.5+ GB


### Preliminary Feature Extraction

Let's first extract some useful information from tpep_pickup_datetime. Things such as day of the week, day of month, month, and trip duration.

In [10]:
#Create a column for the clean date.
yel2019['clean_pickup_datetime'] = pd.to_datetime(yel2019['tpep_pickup_datetime'], format= '%Y-%m-%d %H:%M:%S')
yel2019['clean_dropoff_datetime'] = pd.to_datetime(yel2019['tpep_dropoff_datetime'], format= '%Y-%m-%d %H:%M:%S')

In [11]:
#Extract day of the week, day of month, and month.
pickup = yel2019['clean_pickup_datetime']
dropoff = yel2019['clean_dropoff_datetime']
yel2019['month'] = pickup.dt.month
yel2019['dom'] = pickup.dt.day
yel2019['dow'] = pickup.dt.dayofweek

In [12]:
#Extract the duration of the trip.
yel2019['diff_sec'] = (dropoff-pickup)/np.timedelta64(1,'s')

In [13]:
#Drop the unecessary columns.
yel2019 = yel2019.drop(['tpep_pickup_datetime','tpep_dropoff_datetime'],axis=1)

In [14]:
yel2019.head(10)

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,...,tip_amount,tolls_amount,improvement_surcharge,total_amount,clean_pickup_datetime,clean_dropoff_datetime,month,dom,dow,diff_sec
0,2,2,1.10,-73.990372,40.734695,1,N,-73.981842,40.732407,2,...,0.0,0.0,0.3,8.8,2016-01-01 00:00:00,2016-01-01 00:00:00,1,1,4,0.0
1,2,5,4.90,-73.980782,40.729912,1,N,-73.944473,40.716679,1,...,0.0,0.0,0.3,19.3,2016-01-01 00:00:00,2016-01-01 00:00:00,1,1,4,0.0
2,2,1,10.54,-73.984550,40.679565,1,N,-73.950272,40.788925,1,...,0.0,0.0,0.3,34.3,2016-01-01 00:00:00,2016-01-01 00:00:00,1,1,4,0.0
3,2,1,4.75,-73.993469,40.718990,1,N,-73.962242,40.657333,2,...,0.0,0.0,0.3,17.3,2016-01-01 00:00:00,2016-01-01 00:00:00,1,1,4,0.0
4,2,3,1.76,-73.960625,40.781330,1,N,-73.977264,40.758514,2,...,0.0,0.0,0.3,8.8,2016-01-01 00:00:00,2016-01-01 00:00:00,1,1,4,0.0
5,2,2,5.52,-73.980118,40.743050,1,N,-73.913490,40.763142,2,...,0.0,0.0,0.3,20.3,2016-01-01 00:00:00,2016-01-01 00:18:30,1,1,4,1110.0
6,2,2,7.45,-73.994057,40.719990,1,N,-73.966362,40.789871,2,...,0.0,0.0,0.3,27.3,2016-01-01 00:00:00,2016-01-01 00:26:45,1,1,4,1605.0
7,1,1,1.20,-73.979424,40.744614,1,N,-73.992035,40.753944,2,...,0.0,0.0,0.3,10.3,2016-01-01 00:00:01,2016-01-01 00:11:55,1,1,4,714.0
8,1,1,6.00,-73.947151,40.791046,1,N,-73.920769,40.865578,2,...,0.0,0.0,0.3,19.3,2016-01-01 00:00:02,2016-01-01 00:11:14,1,1,4,672.0
9,2,1,3.21,-73.998344,40.723896,1,N,-73.995850,40.688400,2,...,0.0,0.0,0.3,12.8,2016-01-01 00:00:02,2016-01-01 00:11:08,1,1,4,666.0


### Converting Lat/Long Into Meaningful Features

In [28]:
yel2019.describe()

,VendorID,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,month,dom,dow,diff_sec
count,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00,10906858.00
mean,1.54,1.67,4.65,-72.82,40.11,1.04,-72.89,40.15,1.35,12.49,0.31,0.50,1.75,0.29,0.30,15.64,1.00,15.99,3.16,916.13
std,0.50,1.32,2981.10,9.17,5.05,0.52,8.90,4.90,0.49,35.56,0.42,0.05,2.62,1.69,0.01,36.41,0.00,8.88,1.94,5214.43
min,1.00,0.00,0.00,-121.93,0.00,1.00,-121.93,0.00,1.00,-957.60,-42.61,-0.50,-220.80,-17.40,-0.30,-958.40,1.00,1.00,0.00,-2637.00
25%,1.00,1.00,1.00,-73.99,40.74,1.00,-73.99,40.73,1.00,6.50,0.00,0.50,0.00,0.00,0.30,8.30,1.00,9.00,2.00,380.00
50%,2.00,1.00,1.67,-73.98,40.75,1.00,-73.98,40.75,1.00,9.00,0.00,0.50,1.26,0.00,0.30,11.62,1.00,16.00,3.00,628.00
75%,2.00,2.00,3.08,-73.97,40.77,1.00,-73.96,40.77,2.00,14.00,0.50,0.50,2.32,0.00,0.30,17.16,1.00,23.00,5.00,1013.00
max,2.00,9.00,8000010.00,0.00,60.91,99.00,0.00,60.91,5.00,111270.85,648.87,89.70,998.14,980.15,0.30,111271.65,1.00,31.00,6.00,7194762.00


In [54]:
g = geocoder.google([round(yel2019['pickup_longitude'][0],2), round(yel2019['pickup_latitude'][0],2)], method='reverse')

In [56]:
print(g.state)

None


In [51]:
[round(yel2019['pickup_longitude'][0],4), round(yel2019['pickup_latitude'][0],4)]

[-73.9904, 40.7347]

In [68]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")
location = geolocator.reverse(str(round(yel2019['pickup_longitude'][0],4)) + ', ' + str(round(yel2019['pickup_latitude'][0],4)))

In [67]:
str(round(yel2019['pickup_longitude'][0],4)) + ', ' + str(round(yel2019['pickup_latitude'][0],4))

'-73.9904, 40.7347'

In [69]:
location

Location(None, (0.0, 0.0, 0.0))

### Preliminary Data Cleaning

In [ ]:
#Drop some unecesssary columns.
taxi_merged = taxi_merged.drop(['VendorID',
                                'PULocationID',
                                'DOLocationID',
                                'clean_pickup_datetime',
                                'clean_dropoff_datetime'],axis=1)

In [ ]:
#Convert some columns into factors.
factors = ['RatecodeID','payment_type','month','dom','dow']
for i in factors:
    taxi_merged[i] = taxi_merged[i].astype(object)

In [ ]:
#Check out the numerical columns.
pd.options.display.float_format = "{:.2f}".format
taxi_merged.describe()

There are some oddities in this data. Let's do some cleanup.

In [ ]:
taxi_merged = taxi_merged[taxi_merged['month']==1]
taxi_merged = taxi_merged[taxi_merged['passenger_count']>0]
taxi_merged = taxi_merged[taxi_merged['fare_amount']>0]
taxi_merged = taxi_merged[taxi_merged['tip_amount']>=0]
taxi_merged = taxi_merged[taxi_merged['diff_sec']>0]

In [ ]:
taxi_merged.describe()

Looks a lot better except for the extreme outliers. Let's remove all rows that are greater than the 99th percentile.

In [ ]:
#This is lazy coding - sorry!
numerical_cols = taxi_merged.describe().columns
for cols in numerical_cols:
    taxi_merged = taxi_merged[taxi_merged[cols]<=taxi_merged[cols].quantile(q=.99)]

In [ ]:
taxi_merged.describe()

In [ ]:
#Any missing values?
taxi_merged.isnull().sum()

### Preliminary EDA

Goal: Propose a metric or algorithm to assess the potential efficiency of aggregating rides 
from many vehicles into one, given the available data. Make realistic assumptions and any necessary simplifications and state them.

In [ ]:
taxi_merged.head(10)

One of the first thoughts that comes to mind are single unit passengers who hail a taxi in the same time and same location of another single unit passenger.

In [ ]:
#What is the distribution of passenger counts?
sb.countplot(taxi_merged['passenger_count'])

A majority of taxi rides are single passenger rides.

In [ ]:
taxi_merged['DOZone'].nunique()